<a href="https://colab.research.google.com/github/2b14-1936179-nyikhantmoe/bowling-a2c/blob/main/bowling_dqn_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

     |████████████████████████████████| 320.4MB 37kB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 20.1MB 11.4MB/s 
     |████████████████████████████████| 460kB 39.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
import gym 
import random

In [ ]:
env = gym.make('Bowling-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'UP', 'DOWN', 'UPFIRE', 'DOWNFIRE']

In [ ]:
%%bash
# install required system dependencies
apt-get install -y xvfb x11-utils
# install required python dependencies
pip install gym[box2d]==0.17.* \
            pyvirtualdisplay==0.2.* \
            PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.*
# install the gym
pip install gym-notebook-wrapper

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


In [ ]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:19.0
Episode:2 Score:30.0
Episode:3 Score:21.0
Episode:4 Score:16.0
Episode:5 Score:21.0


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
 2769/10000: episode: 1, duration: 192.717s, episode steps: 2769, steps per second:  14, episode reward: 32.000, mean reward:  0.012 [ 0.000,  7.000], mean action: 2.524 [0.000, 5.000],  loss: 30.664276, mean_q: 11.326173, mean_eps: 0.830395
 5598/10000: episode: 2, duration: 285.307s, episode steps: 2829, steps per second:  10, episode reward: 40.000, mean reward:  0.014 [ 0.000,  7.000], mean action: 2.414 [0.000, 5.000],  loss: 0.119072, mean_q: 10.332195, mean_eps: 0.623530
 8435/10000: episode: 3, duration: 283.736s, episode steps: 2837, steps per second:  10, episode reward: 12.000, mean reward:  0.004 [ 0.000,  3.000], mean action: 2.441 [0.000, 5.000],  loss: 0.062950, mean_q: 10.413612, mean_eps: 0.368560
done, took 917.240 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 12.000, steps: 10000
Episode 2: reward: 6.000, steps: 10000
Episode 3: reward: 12.000, steps: 10000
Episode 4: reward: 6.000, steps: 10000
Episode 5: reward: 12.000, steps: 10000
Episode 6: reward: 6.000, steps: 10000
Episode 7: reward: 3.000, steps: 10000
Episode 8: reward: 12.000, steps: 10000
Episode 9: reward: 15.000, steps: 10000
Episode 10: reward: 3.000, steps: 10000
8.7


### Total Score: 87

In [ ]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 15.000, steps: 10000
Episode 2: reward: 9.000, steps: 10000
Episode 3: reward: 12.000, steps: 10000
Episode 4: reward: 12.000, steps: 10000
Episode 5: reward: 12.000, steps: 10000
Episode 6: reward: 3.000, steps: 10000
Episode 7: reward: 12.000, steps: 10000
Episode 8: reward: 12.000, steps: 10000
Episode 9: reward: 12.000, steps: 10000
Episode 10: reward: 9.000, steps: 10000
10.8


### Total Score: 108

This model yielded decent scores but there is a problem with dislaying the game and the video. Hence this model is not used for the final submission.

In [1]:
%%shell
jupyter nbconvert --to html /content/bowling_dqn_agent.ipynb

[NbConvertApp] Converting notebook /content/bowling_dqn_agent.ipynb to html
/usr/local/lib/python2.7/dist-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) [u'application/vnd.colab-display-data+json'] is not able to be represented.
  mimetypes=output.keys())
[NbConvertApp] Writing 315052 bytes to /content/bowling_dqn_agent.html
